In [50]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
# pandas display
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.precision", 3)
# plot setting
plt.style.use('ggplot')
%matplotlib inline
import folium
import patsy
import statsmodels.api as sm

# get data

In [41]:
# the csv from www.import.io. manhattan.csv
path = ''
with open(path + 'manhattan.csv','r') as f:
    data = f.read()
    data = data.replace(';',',')
    data = data.replace('\"','')
with open(path + 'manhattan.csv','w') as f:
    f.write(data)

In [65]:
# read csv
df = pd.read_csv('manhattan.csv',skiprows=1,names=list('abcdefghijklm'))
df.head()

In [84]:
# the html contain the single website. html-div
with open(path + 'html-div','r') as f:
    html_div = f.read()

In [85]:
# re
re_txt = re.compile(r"""
(?<=href\=")
/homedetails/
.*?
(?=\")
""",re.VERBOSE)

In [86]:
# get websites
txt_findall=re_txt.findall(html_div)
txt_findall[:5]

['/homedetails/246-E-10th-St-TWO-New-York-NY-10009/2089436092_zpid/',
 '/homedetails/185-E-3rd-St-APT-4D-New-York-NY-10009/2125494760_zpid/',
 '/homedetails/319-E-105th-St-APT-5C-New-York-NY-10029/31546761_zpid/',
 '/homedetails/213-W-21st-St-B-New-York-NY-10011/2089436713_zpid/',
 '/homedetails/128-W-13th-St-APT-42-New-York-NY-10011/2089437698_zpid/']

In [90]:
# save websites
with open(path + 'urls.csv','w') as f:
    for x in txt_findall:
        f.write('https://www.zillow.com/'+x+'\n')

In [91]:
# the csv from www.import.io. manhattan.csv. manhattan1.csv
path = ''
with open(path + 'manhattan1.csv','r') as f:
    data = f.read()

In [2]:
# got data!
df = pd.read_csv('manhattan1.csv')
df = df.iloc[:,1:]

In [3]:
df.head(5)

,price,rooms,addr
0,"$2,495",2 beds 1 bath -- sqft,"185 E 3rd St APT 4D, New York, NY 10009"
1,"$2,800",2 beds 1 bath -- sqft,"319 E 105th St APT 5C, New York, NY 10029"
2,"$3,000",Studio 1 bath 500 sqft,"213 W 21st St # B, New York, NY 10011"
3,"$2,695",Studio 1 bath -- sqft,"128 W 13th St APT 42, New York, NY 10011"
4,"$2,150",Studio 1 bath -- sqft,"308 E 89th St # 3BB, New York, NY 10128"


# deal with data

In [5]:
print(df[~(df['rooms'].str.contains('Studio')|df['rooms'].str.contains('bed'))])
print(df[~df['rooms'].str.contains('bath')])
print('None means rooms column always contain (Studio/bed) and bath')

Empty DataFrame
Columns: [price, rooms, addr]
Index: []
Empty DataFrame
Columns: [price, rooms, addr]
Index: []
为空则说明rooms列中总是包含(Studio/bed)和bath


In [6]:
def dealrooms(cell):
    *beds,n,bath,area,sqft = cell.split(' ')
    beds = " ".join(beds)
    return pd.Series({'beds':beds,'bath':n,'sqft':area})
attr =  df['rooms'].apply(dealrooms)
attr.head()

,bath,beds,sqft
0,1,2 beds,--
1,1,2 beds,--
2,1,Studio,500
3,1,Studio,--
4,1,Studio,--


In [7]:
attr['beds'] = attr['beds'].apply(lambda x: x.split(' ')[0])
attr.head()

,bath,beds,sqft
0,1,2,--
1,1,2,--
2,1,Studio,500
3,1,Studio,--
4,1,Studio,--


In [8]:
attr['sqft'] = attr['sqft'].replace('--',np.nan)
attr.head()

,bath,beds,sqft
0,1,2,NaN
1,1,2,NaN
2,1,Studio,500
3,1,Studio,NaN
4,1,Studio,NaN


In [9]:
df = df.join(attr)

In [10]:
df['price'] = df['price'].apply(lambda x : x[1:])
df.head()

,price,rooms,addr,bath,beds,sqft
0,"2,495",2 beds 1 bath -- sqft,"185 E 3rd St APT 4D, New York, NY 10009",1,2,NaN
1,"2,800",2 beds 1 bath -- sqft,"319 E 105th St APT 5C, New York, NY 10029",1,2,NaN
2,"3,000",Studio 1 bath 500 sqft,"213 W 21st St # B, New York, NY 10011",1,Studio,500
3,"2,695",Studio 1 bath -- sqft,"128 W 13th St APT 42, New York, NY 10011",1,Studio,NaN
4,"2,150",Studio 1 bath -- sqft,"308 E 89th St # 3BB, New York, NY 10128",1,Studio,NaN


In [11]:
def dealadrr(cell):
    so_zip = re.search(', NY (\d+)', cell)
    so_flr = re.search('(?:APT|#)\s+(\d+)[A-Z]+,', cell)
    zipc = so_zip.group(1) if so_zip else np.nan
    flrc = so_flr.group(1) if so_flr else np.nan
    return pd.Series({'zip':zipc,'floor':flrc})
flrzip = df['addr'].apply(dealadrr)
flrzip.head()

,floor,zip
0,4,10009
1,5,10029
2,NaN,10011
3,NaN,10011
4,3,10128


In [12]:
df = df.join(flrzip)

In [13]:
# : deep-copy
house = df[['price','beds','bath','sqft','floor','zip']][:]
house.head()

,price,beds,bath,sqft,floor,zip
0,"2,495",2,1,NaN,4,10009
1,"2,800",2,1,NaN,5,10029
2,"3,000",Studio,1,500,NaN,10011
3,"2,695",Studio,1,NaN,NaN,10011
4,"2,150",Studio,1,NaN,3,10128


# analysis data

In [15]:
house['beds'].replace('Studio',0,inplace=True)
house.head()

,price,beds,bath,sqft,floor,zip
0,"2,495",2,1,NaN,4,10009
1,"2,800",2,1,NaN,5,10029
2,"3,000",0,1,500,NaN,10011
3,"2,695",0,1,NaN,NaN,10011
4,"2,150",0,1,NaN,3,10128


In [16]:
house.describe()

,price,beds,bath,sqft,floor,zip
count,114,114,114,35,79,114
unique,56,3,2,21,13,30
top,"2,600",1,1,500,3,10028
freq,6,55,112,6,18,11


In [17]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 6 columns):
price    114 non-null object
beds     114 non-null object
bath     114 non-null object
sqft     35 non-null object
floor    79 non-null object
zip      114 non-null object
dtypes: object(6)
memory usage: 5.4+ KB


In [18]:
house['price'] = house['price'].str.replace(',','')

In [19]:
house['price'] = house['price'].astype(int)
house['beds'] = house['beds'].astype(int)
house['bath'] = house['bath'].astype(int)

In [20]:
house['sqft'] = house['sqft'].astype(float)
house['floor'] = house['floor'].astype(float)

In [21]:
house['zip'] = house['zip'].astype(str)

In [22]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 6 columns):
price    114 non-null int64
beds     114 non-null int64
bath     114 non-null int64
sqft     35 non-null float64
floor    79 non-null float64
zip      114 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 5.4+ KB


In [23]:
house.head()

,price,beds,bath,sqft,floor,zip
0,2495,2,1,NaN,4.0,10009
1,2800,2,1,NaN,5.0,10029
2,3000,0,1,500.0,NaN,10011
3,2695,0,1,NaN,NaN,10011
4,2150,0,1,NaN,3.0,10128


In [24]:
# pivot
house.pivot_table('price', 'zip', 'beds', aggfunc='mean').head()

beds,0,1,2
zip,,,
10001,2377.500,NaN,NaN
10002,NaN,2481.333,2597.5
10003,2230.000,2690.000,2849.5
10009,NaN,2572.500,2495.0
10010,2308.333,NaN,NaN


In [25]:
house.pivot_table('price', 'zip', 'beds', aggfunc='count').head()

beds,0,1,2
zip,,,
10001,2.0,NaN,NaN
10002,NaN,3.0,4.0
10003,3.0,5.0,2.0
10009,NaN,2.0,1.0
10010,3.0,NaN,NaN


In [51]:
house = house[house['beds']<2]

# price maps

In [47]:
# map = folium.Map(location=[40.748817, -73.985428], zoom_start=13)
# # nyc.json来源: http://data.beta.nyc//dataset/3bf5fb73-edb5-4b05-bb29-7c95f4a727fc/resource/6df127b1-6d04-4bb7-b983-07402a2c3f90/download/f4129d9aa6dd4281bc98d0f701629b76nyczipcodetabulationareas.geojson
# map.choropleth(geo_path=r'nyc.json', 
#                data=house,
#                columns=['zip','price'] ,
#                key_on='feature.properties.postalCode',
#                threshold_scale=[1700.00, 1900.00, 2100.00, 2300.00, 2500.00, 2750.00],
#                fill_color='', 
#                fill_opacity=0.7, 
#                line_opacity=0.2,
#                legend_name='price (%)',
#                reset=True)
# map

# modeling

In [55]:
f = 'price ~ zip + beds'
y, X = patsy.dmatrices(f, house, return_type='dataframe')
# dummy varibale

In [57]:
# linear reg
results =sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.364
Method:                 Least Squares   F-statistic:                     2.736
Date:                Thu, 24 May 2018   Prob (F-statistic):           0.000526
Time:                        12:49:37   Log-Likelihood:                -619.02
No. Observations:                  89   AIC:                             1298.
Df Residuals:                      59   BIC:                             1373.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2377.5000    220.380     10.788   

# predict

In [58]:
X.head()

,Intercept,zip[T.10002],zip[T.10003],zip[T.10009],zip[T.10010],zip[T.10011],zip[T.10012],zip[T.10013],zip[T.10014],zip[T.10016],zip[T.10018],zip[T.10019],zip[T.10021],zip[T.10022],zip[T.10023],zip[T.10024],zip[T.10025],zip[T.10026],zip[T.10027],zip[T.10028],zip[T.10029],zip[T.10030],zip[T.10031],zip[T.10032],zip[T.10036],zip[T.10065],zip[T.10075],zip[T.10128],zip[T.10280],beds
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [73]:
to_pred_idx = X.iloc[0].index
to_pred_zeros = np.zeros(len(to_pred_idx))
tpdf = pd.DataFrame(to_pred_zeros, index = to_pred_idx, columns = ['value'])
tpdf.iloc[[0,1,2,-1]]

,value
Intercept,0.0
zip[T.10002],0.0
zip[T.10003],0.0
beds,0.0


In [79]:
# predict1 
tpdf.loc['Intercept'] = 1
tpdf.loc['beds'] = 1
tpdf.loc['zip[T.10009]'] = 1
results.predict(tpdf['value'])[0]

2572.4999999999964

In [80]:
# predict2
tpdf['value'] = 0
tpdf.loc['Intercept'] = 1
tpdf.loc['beds'] = 2
tpdf.loc['zip[T.10009]'] = 1
results.predict(tpdf['value'])[0]

2863.067395018429

In [81]:
# predict3
tpdf['value'] = 0
tpdf.loc['Intercept'] = 1
tpdf.loc['beds'] = 2
tpdf.loc['zip[T.10002]'] = 1
results.predict(tpdf['value'])[0]

2771.9007283517603